In [1]:
install.packages("GGally")
install.packages("corrplot")  
install.packages("themis")

Warning message in install.packages("GGally"):
“installation of package ‘GGally’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [2]:
library(repr)
library(tidyverse)
library(tidymodels)
library(RColorBrewer)
options(repr.matrix.max.rows = 6)
library(GGally)       
library(corrplot)
library(cowplot)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

ERROR: Error in library(GGally): there is no package called ‘GGally’


In [68]:
winequalityRed <- read_csv2("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv")|> 
                    mutate(quality = as.factor(quality))
head(winequalityRed)



ℹ Using "','" as decimal and "'.'" as grouping mark. Use `read_delim()` for more control.

Warning message:
“One or more parsing issues, see `problems()` for details”
Rows: 1599 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr (5): volatile acidity, citric acid, chlorides, density, sulphates
dbl (2): total sulfur dioxide, quality

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
<dbl>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<fct>
74,0.7,0,19,0.076,11,34,0.9978,351,0.56,94,5
78,0.88,0,26,0.098,25,67,0.9968,32,0.68,98,5
78,0.76,0.04,23,0.092,15,54,0.997,326,0.65,98,5
112,0.28,0.56,19,0.075,17,60,0.998,316,0.58,98,6
74,0.7,0,19,0.076,11,34,0.9978,351,0.56,94,5
74,0.66,0,18,0.075,13,40,0.9978,351,0.56,94,5


In [69]:
completedt <- winequalityRed |>
              setNames(c("fixed_acidity",
                         "volatile_acidity",
                         "citric_acid",
                         "residual_sugar",
                         "chlorides",
                         "free_SO2",
                         "total_SO2",
                         "density",
                         "pH",
                         "sulphates",
                         "alcohol",
                         "quality",
                         "type"))
completedt$volatile_acidity <- as.numeric(completedt$volatile_acidity)
completedt$citric_acid <- as.numeric(completedt$citric_acid)
completedt$residual_sugar <- as.numeric(completedt$residual_sugar)
completedt$chlorides <- as.numeric(completedt$chlorides)
completedt$density <- as.numeric(completedt$density)
completedt$sulphates <- as.numeric(completedt$sulphates)
head(completedt)

fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_SO2,total_SO2,density,pH,sulphates,alcohol,quality
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
74,0.70,0.00,19,0.076,11,34,0.9978,351,0.56,94,5
78,0.88,0.00,26,0.098,25,67,0.9968,32,0.68,98,5
78,0.76,0.04,23,0.092,15,54,0.9970,326,0.65,98,5
112,0.28,0.56,19,0.075,17,60,0.9980,316,0.58,98,6
74,0.70,0.00,19,0.076,11,34,0.9978,351,0.56,94,5
74,0.66,0.00,18,0.075,13,40,0.9978,351,0.56,94,5


In [92]:
completedt<- completedt|>
                     mutate_at(c('quality'), as.numeric)|>
                     filter(!is.na(total_SO2))

head(completedt)

fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_SO2,total_SO2,density,pH,sulphates,alcohol,quality
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
74,0.70,0.00,19,0.076,11,34,0.9978,351,0.56,94,3
78,0.88,0.00,26,0.098,25,67,0.9968,32,0.68,98,3
78,0.76,0.04,23,0.092,15,54,0.9970,326,0.65,98,3
112,0.28,0.56,19,0.075,17,60,0.9980,316,0.58,98,4
74,0.70,0.00,19,0.076,11,34,0.9978,351,0.56,94,3
74,0.66,0.00,18,0.075,13,40,0.9978,351,0.56,94,3


In [93]:
set.seed(100)
wine_split <- initial_split(completedt, prop = 0.75, strata = quality)
wine_training <- training(wine_split)
wine_testing <- testing(wine_split)
head(wine_training)

fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_SO2,total_SO2,density,pH,sulphates,alcohol,quality
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
74,0.70,0.00,19,0.076,11,34,0.9978,351,0.56,94,3
78,0.88,0.00,26,0.098,25,67,0.9968,32,0.68,98,3
74,0.70,0.00,19,0.076,11,34,0.9978,351,0.56,94,3
79,0.60,0.06,16,0.069,15,59,0.9964,33,0.46,94,3
75,0.50,0.36,61,0.071,17,102,0.9978,335,0.80,105,3
67,0.58,0.08,18,0.097,15,65,0.9959,328,0.54,92,3


In [119]:
wine_recipe <- recipe(volatile_acidity ~ citric_acid + chlorides + pH + sulphates + total_SO2 + alcohol +fixed_acidity , data = wine_training) |>
  step_scale(all_predictors()) |>
  step_center(all_predictors())

wine_spec <- nearest_neighbor(weight_func = "rectangular", 
                              neighbors = tune()) |>
  set_engine("kknn") |>
  set_mode("regression")

wine_vfold <- vfold_cv(wine_training, v = 5, strata = volatile_acidity)

wine_wkflw <- workflow() |>
  add_recipe(wine_recipe) |>
  add_model(wine_spec)

In [120]:
gridvals <- tibble(neighbors = seq(from = 1, to = 200, by = 3))

wine_results <- wine_wkflw |>
  tune_grid(resamples = wine_vfold, grid = gridvals) |>
  collect_metrics() |>
  filter(.metric == "rmse")

wine_min <- wine_results |>
  filter(mean == min(mean)) 

wine_min

neighbors,.metric,.estimator,mean,n,std_err,.config
<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
7,rmse,standard,0.1383765,5,0.00359566,Preprocessor1_Model03


In [122]:
wine_spec <- nearest_neighbor(weight_func = "rectangular", 
                              neighbors = 7) |>
  set_engine("kknn") |>
  set_mode("regression")

knn_mult_fit <- workflow() |>
  add_recipe(wine_recipe) |>
  add_model(wine_spec) |>
  fit(data = wine_training)

knn_mult_preds <- knn_mult_fit |>
  predict(wine_testing) |>
 bind_cols(wine_testing) |>
    select(.pred, volatile_acidity)

head(knn_mult_preds)

knn_mult_mets <- metrics(knn_mult_preds, truth = volatile_acidity, estimate = .pred) |>
                     filter(.metric == 'rmse')

knn_mult_mets

.pred,volatile_acidity
<dbl>,<dbl>
0.5871429,0.760
0.6650000,0.660
0.7157143,0.615
0.4614286,0.430
0.5164286,0.410
0.6042857,0.710


.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
rmse,standard,0.126932
